# Learning Doc2vec

In [20]:
import pickle
from IPython.display import display
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.models import doc2vec
from platform import python_version
import random
import numpy as np
from tqdm import tqdm
from pandas import DataFrame
import copy
import ast
import pandas as pd
from math import log

# Upload data

In [27]:
pone = pd.read_csv('../../Data/Experiment/Plosone/pone_final.csv')
pone.head()

,Unnamed: 0,Year,Title,Abstract,Role,Affiliation index,Affiliation list,Discipline,Filename
0,0,2016,"1,25(OH)2D3 and VDR Signaling Pathways Regulat...",\n\nBackground\nThe objective of this study is...,"[[], [], [], [], [], [], [], []]","[['aff001'], ['aff001', 'cor001'], ['aff001'],...","['1 Department of Ophthalmology, The Affiliate...","['Medicine and health sciences', 'Anatomy', 'B...",/data02/plos/pone/journal.pone.0164717.xml
1,1,2018,"A non-invasive, quantitative study of broadban...","\nCurrently, non-invasive methods for studying...","[['Data curation', 'Formal analysis', 'Investi...","[['aff001', 'cor001'], ['aff001'], ['aff002'],...",['1 Department of Psychology and Center for Ne...,"['Engineering and technology', 'Signal process...",/data02/plos/pone/journal.pone.0193107.xml
2,2,2016,The Effects of Financial Education on Impulsiv...,"\nDelay discounting, as a behavioral measure o...","[[], [], [], []]","[['aff001', 'cor001'], ['aff001'], ['aff002'],...","['1 Department of Psychology, Utah State Unive...","['Social sciences', 'Psychology', 'Personality...",/data02/plos/pone/journal.pone.0159561.xml
3,3,2016,Root-Zone Warming Differently Benefits Mature ...,\nSub-optimal temperature extensively suppress...,"[[], [], [], []]","[['aff001'], ['aff001'], ['aff001'], ['cor001'...",['Beijing Key Laboratory of Growth and Develop...,"['Medicine and health sciences', 'Vascular med...",/data02/plos/pone/journal.pone.0155298.xml
4,4,2017,A hypomorphic PIGA gene mutation causes severe...,\nMutations in genes involved in glycosylphosp...,"[[], [], [], [], [], [], [], [], [], [], []]","[['aff001'], ['aff002'], ['aff001'], ['aff001'...","['1 Division of Hematology, Department of Medi...","['Biology and life sciences', 'Cell biology', ...",/data02/plos/pone/journal.pone.0174074.xml


In [29]:
pone.loc[18]['Discipline']

"['Social sciences', 'Psychology', 'Cognitive psychology', 'Reasoning']"

In [3]:
import ast

In [6]:
pone_tfidf= pd.read_csv('../../Data/Experiment/Plosone/pone_tfidf0_01.csv')


0        [background, object, studi, observ, whether, c...
1        [current, non, invas, method, studi, human, br...
2        [delay, discount, behavior, measur, impuls, ch...
3        [sub, optim, temperatur, extens, suppress, cro...
4        [mutat, gene, involv, glycosylphosphatidylinos...
                               ...                        
70382    [organ, mix, mode, data, collect, self, admini...
70383    [paper, andriessen, al, present, valid, fetal,...
70384    [number, undocu, immigr, unit, state, estim, b...
70385    [conclus, recent, paper, perform, stan, devic,...
70386    [introduct, intern, extern, qualiti, control, ...
Name: Total, Length: 70387, dtype: object

In [9]:
pone_tfidf['Total_tokenize'] = pone_tfidf['Total_tokenize'].apply(lambda x: ast.literal_eval(x))

In [10]:
pone_tfidf

,Unnamed: 0,Year,Title,Abstract,Role,Affiliation index,Affiliation list,Discipline,Filename,Total,Total_tokenize
0,0,2016,"['oh', 'vdr', 'signal', 'pathway', 'regul', 'i...","['background', 'object', 'studi', 'observ', 'w...","[[], [], [], [], [], [], [], []]","[['aff001'], ['aff001', 'cor001'], ['aff001'],...","['1 Department of Ophthalmology, The Affiliate...","['Medicine and health sciences', 'Anatomy', 'B...",/data02/plos/pone/journal.pone.0164717.xml,"['background', 'object', 'studi', 'observ', 'w...","[background, object, observ, whether, cyclospo..."
1,1,2018,"['non', 'invas', 'quantit', 'studi', 'broadban...","['current', 'non', 'invas', 'method', 'studi',...","[['Data curation', 'Formal analysis', 'Investi...","[['aff001', 'cor001'], ['aff001'], ['aff002'],...",['1 Department of Psychology and Center for Ne...,"['Engineering and technology', 'Signal process...",/data02/plos/pone/journal.pone.0193107.xml,"['current', 'non', 'invas', 'method', 'studi',...","[current, non, invas, method, studi, human, br..."
2,2,2016,"['effect', 'financi', 'educ', 'impuls', 'decis...","['delay', 'discount', 'behavior', 'measur', 'i...","[[], [], [], []]","[['aff001', 'cor001'], ['aff001'], ['aff002'],...","['1 Department of Psychology, Utah State Unive...","['Social sciences', 'Psychology', 'Personality...",/data02/plos/pone/journal.pone.0159561.xml,"['delay', 'discount', 'behavior', 'measur', 'i...","[delay, discount, behavior, measur, impuls, ch..."
3,3,2016,"['root', 'zone', 'warm', 'differ', 'benefit', ...","['sub', 'optim', 'temperatur', 'extens', 'supp...","[[], [], [], []]","[['aff001'], ['aff001'], ['aff001'], ['cor001'...",['Beijing Key Laboratory of Growth and Develop...,"['Medicine and health sciences', 'Vascular med...",/data02/plos/pone/journal.pone.0155298.xml,"['sub', 'optim', 'temperatur', 'extens', 'supp...","[sub, optim, temperatur, extens, suppress, cro..."
4,4,2017,"['hypomorph', 'piga', 'gene', 'mutat', 'caus',...","['mutat', 'gene', 'involv', 'glycosylphosphati...","[[], [], [], [], [], [], [], [], [], [], []]","[['aff001'], ['aff002'], ['aff001'], ['aff001'...","['1 Division of Hematology, Department of Medi...","['Biology and life sciences', 'Cell biology', ...",/data02/plos/pone/journal.pone.0174074.xml,"['mutat', 'gene', 'involv', 'glycosylphosphati...","[mutat, gene, involv, glycosylphosphatidylinos..."
...,...,...,...,...,...,...,...,...,...,...,...
70382,70382,2018,"['measur', 'agreement', 'self', 'administ', 'q...","['organ', 'mix', 'mode', 'data', 'collect', 's...","[['Formal analysis', 'Investigation', 'Methodo...","[['aff001', 'aff002', 'cor001'], ['aff001'], [...","['1 Department Epidemiology and public health,...","['Medicine and health sciences', 'Mental healt...",/data02/plos/pone/journal.pone.0197434.xml,"['organ', 'mix', 'mode', 'data', 'collect', 's...","[organ, mix, mode, data, collect, self, admini..."
70383,70383,2019,"['st', 'analysi', 'fetal', 'electrocardiogram'...","['paper', 'andriessen', 'al', 'present', 'vali...","[['Conceptualization', 'Formal analysis', 'Wri...","[['aff001'], ['aff002'], ['aff003', 'currentaf...","['1 Department of Pediatrics, Institute of Cli...","['Medicine and health sciences', 'Cardiology',...",/data02/plos/pone/journal.pone.0221210.xml,"['paper', 'andriessen', 'al', 'present', 'vali...","[paper, andriessen, al, present, valid, fetal,..."
70384,70384,2018,"['commentari', 'number', 'undocu', 'immigr', '...","['number', 'undocu', 'immigr', 'unit', 'state'...","[['Writing – original draft'], ['Methodology',...","[['aff001'], ['aff001'], ['aff002'], ['aff001'...","['1 Migration Policy Institute, Washington, D....","['Social sciences', 'Human geography', 'Housing']",/data02/plos/pone/journal.pone.0204199.xml,"['number', 'undocu', 'immigr', 'unit', 'state'...","[number, undocu, immigr, unit, state, estim, b..."
70385,70385,2019,"['avf', 'use', 'stan', 'proxi', 'scalp', 'elec...","['conclus', 'recent', 'paper', 'perform', 'sta...","[['Conceptualization', 'Data curation', '

## Doc2vec

In [11]:
all_tagged_data = [TaggedDocument(words=_d, tags=[uid]) for uid, _d in enumerate(pone_tfidf['Total_tokenize'])]

In [12]:
all_tagged_data[1]

TaggedDocument(words=['current', 'non', 'invas', 'method', 'studi', 'human', 'brain', 'routin', 'reliabl', 'measur', 'spike', 'rate', 'depend', 'signal', 'independ', 'respons', 'hemodynam', 'coupl', 'fmri', 'subthreshold', 'neuron', 'synchroni', 'oscil', 'event', 'relat', 'potenti', 'contrast', 'invas', 'method', 'microelectrod', 'record', 'electrocorticographi', 'ecog', 'recent', 'measur', 'broadband', 'power', 'elev', 'field', 'potenti', 'hz', 'proxi', 'local', 'averag', 'spike', 'rate', 'sought', 'detect', 'quantifi', 'stimulu', 'relat', 'broadband', 'respons', 'use', 'magnetoencephalographi', 'meg', 'extracrani', 'measur', 'like', 'meg', 'eeg', 'multipl', 'global', 'nois', 'sourc', 'rel', 'low', 'signal', 'nois', 'ratio', 'moreov', 'high', 'frequenc', 'artifact', 'eye', 'movement', 'confound', 'stimulu', 'design', 'mistaken', 'signal', 'origin', 'brain', 'activ', 'reason', 'develop', 'autom', 'denois', 'techniqu', 'help', 'reveal', 'broadband', 'signal', 'interest', 'subject', 'vie

In [13]:
model4 = Doc2Vec(size=200,dm =1,seed = 3000,epochs=10) 
model4.build_vocab(all_tagged_data)
model4.train(all_tagged_data,total_examples=model4.corpus_count,epochs=model4.epochs)

/home/balla/anaconda3/lib/python3.7/site-packages/gensim/models/doc2vec.py:574: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


In [15]:
model4.hs

0

### most similar words with 'criminal'(='범죄')

In [16]:
model4.save('../../Data/Experiment/Plosone/doc2vec_model001') #dlrj

In [17]:
model4.wv.vocab

{'background': <gensim.models.keyedvectors.Vocab at 0x7fe730018b50>,
 'object': <gensim.models.keyedvectors.Vocab at 0x7fe6d2346210>,
 'observ': <gensim.models.keyedvectors.Vocab at 0x7fe6d0c66950>,
 'whether': <gensim.models.keyedvectors.Vocab at 0x7fe6a7608590>,
 'cyclosporin': <gensim.models.keyedvectors.Vocab at 0x7fe6a76086d0>,
 'csa': <gensim.models.keyedvectors.Vocab at 0x7fe6a76084d0>,
 'inhibit': <gensim.models.keyedvectors.Vocab at 0x7fe6a7608bd0>,
 'express': <gensim.models.keyedvectors.Vocab at 0x7fe6a76083d0>,
 'dectin': <gensim.models.keyedvectors.Vocab at 0x7fe6a7608550>,
 'human': <gensim.models.keyedvectors.Vocab at 0x7fe6a76087d0>,
 'corneal': <gensim.models.keyedvectors.Vocab at 0x7fe6a7608750>,
 'epitheli': <gensim.models.keyedvectors.Vocab at 0x7fe6a7608f50>,
 'cell': <gensim.models.keyedvectors.Vocab at 0x7fe6bd54c790>,
 'infect': <gensim.models.keyedvectors.Vocab at 0x7fe6aaf23290>,
 'aspergillu': <gensim.models.keyedvectors.Vocab at 0x7fe6b698c1d0>,
 'fumigatu':

In [18]:
model4.wv.most_similar('rabbit')

[('rat', 0.6025505065917969),
 ('mous', 0.5714064836502075),
 ('mice', 0.5233096480369568),
 ('porcin', 0.5095186233520508),
 ('pig', 0.4973297417163849),
 ('minipig', 0.4951893985271454),
 ('orthotop', 0.4941790699958801),
 ('murin', 0.4939144551753998),
 ('nzw', 0.49258679151535034),
 ('spragu', 0.46846479177474976)]

In [19]:
model4.wv.most_similar('happi')

[('anger', 0.667168140411377),
 ('unhappi', 0.6430577039718628),
 ('sad', 0.6390724182128906),
 ('angri', 0.6108818054199219),
 ('valenc', 0.6012066006660461),
 ('joy', 0.5795535445213318),
 ('emot', 0.5770508646965027),
 ('pleasant', 0.5745635032653809),
 ('unpleas', 0.5524196624755859),
 ('feel', 0.5403744578361511)]